In [1]:
from osgeo import gdal, gdalconst, ogr, osr
import numpy as np
import rasterio


In [2]:

num_channel = 8

In [3]:

# img_for_follow = r"/home/skm/SKM16/Work/Npark_planet/image_roi_rac/20220330_023353_67_2460_3B_AnalyticMS_SR_8b_clip.tif"
# img_for_follow = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/20220330_023353_67_2460_3B_AnalyticMS_SR_8b_clip.tif"
img_for_follow = r"/home/skm/SKM16/Work/Npark_planet2/img/view/T4_2022_32648.tif"
dataset = gdal.Open(img_for_follow, gdal.GA_ReadOnly)
dict_band_min_max_percentile_follow = {k: dict(max=0, min=0) for k in range(num_channel)}

for i_chan in range(num_channel):
    band = dataset.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
    band[band==0] = np.nan
    dict_band_min_max_percentile_follow[i_chan]['max'] = np.nanpercentile(band, 98)
    dict_band_min_max_percentile_follow[i_chan]['min'] = np.nanpercentile(band, 2)



In [4]:
# image_path = r"/home/skm/SKM16/Work/Npark_planet/image_roi_rac/giu/20220313_023541_10_2465_3B_AnalyticMS_SR_8b_clip.tif"
import os, glob
dir_img = r"/home/skm/SKM16/Work/Npark_planet2/img/view"
list_fp = glob.glob(os.path.join(dir_img,'*.tif'))
k = 0
for image_path in list_fp:
    # image_path = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/20220330_030416_80_248c_3B_AnalyticMS_SR_8b_clip.tif"
    dataset_ori = gdal.Open(image_path, gdal.GA_ReadOnly)
    dict_band_min_max_percentile_ori = {k: dict(max=0, min=0) for k in range(num_channel)}

    for i_chan in range(num_channel):
        band = dataset_ori.GetRasterBand(i_chan+1).ReadAsArray().astype(np.float16)
        band[band==0] = np.nan
        dict_band_min_max_percentile_ori[i_chan]['max'] = np.nanpercentile(band, 98)
        dict_band_min_max_percentile_ori[i_chan]['min'] = np.nanpercentile(band, 2)
    src = rasterio.open(image_path)
    meta = src.meta

    img_ = src.read()
    img_percentile = np.empty(img_.shape)
    for i in range(8):
        idxnodata = np.where(img_[i]==meta['nodata'])
        # img_[i][img_[i] > dict_band_min_max_percentile[i]['max']] = dict_band_min_max_percentile[i]['max']
        # img_[i][img_[i] < dict_band_min_max_percentile[i]['min']] = dict_band_min_max_percentile[i]['min']
        # img_[i][idxnodata]=meta['nodata']

        img_percentile[i] = np.interp(img_[i], (dict_band_min_max_percentile_ori[i]['min'], dict_band_min_max_percentile_ori[i]['max']), 
                                            (dict_band_min_max_percentile_follow[i]['min'], dict_band_min_max_percentile_follow[i]['max']))
        img_percentile[i][idxnodata]=meta['nodata']
        img_percentile[i].astype('uint16')
    img_percentile = img_percentile.astype('uint16')
    # out = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/mosaic/mosaic3.tif"
    k+=1
    out = os.path.join("/home/skm/SKM16/Work/Npark_planet2/img/view/percentile", f"mosaic_{k}.tif")
    with rasterio.open(out, 'w', **meta) as dst:
        dst.write(img_percentile)
    print("xong")


xong
xong


In [ ]:

# src = rasterio.open(image_path)
# meta = src.meta

# img_ = src.read()
# img_percentile = np.empty(img_.shape)
# for i in range(8):
#     idxnodata = np.where(img_[i]==meta['nodata'])
#     # img_[i][img_[i] > dict_band_min_max_percentile[i]['max']] = dict_band_min_max_percentile[i]['max']
#     # img_[i][img_[i] < dict_band_min_max_percentile[i]['min']] = dict_band_min_max_percentile[i]['min']
#     # img_[i][idxnodata]=meta['nodata']

#     img_percentile[i] = np.interp(img_[i], (dict_band_min_max_percentile_ori[i]['min'], dict_band_min_max_percentile_ori[i]['max']), 
#                                            (dict_band_min_max_percentile_follow[i]['min'], dict_band_min_max_percentile_follow[i]['max']))
#     img_percentile[i][idxnodata]=meta['nodata']
#     img_percentile[i].astype('uint16')




In [ ]:
# img_percentile = img_percentile.astype('uint16')
# out = r"/home/skm/SKM16/Work/Npark_planet/Z_Tat_ca_anh_roi_rac/img_origin_remove_cloud/t3/mosaic/mosaic3.tif"
# with rasterio.open(out, 'w', **meta) as dst:
#     dst.write(img_percentile)
# print("xong")

In [1]:
from mosaic import mosaic1
dir_path = r"/home/skm/SKM16/Work/Npark_planet2/img/view/percentile"
list_img_name = [ "mosaic_2.tif",
                  "mosaic_1.tif" 
]
out_path = r"/home/skm/SKM16/Work/Npark_planet2/img/view/T4/T4_2022_32648_ok_3.tif"
mosaic1(dir_path, list_img_name, out_path)


# Lay nodata anh dau tien

In [4]:
import rasterio
import numpy as np
fp = r"/home/geoai/geoai_data_test2/data_npark_prod_PlanetScope/data/2022/03/images/4439c0917fcd411da7f299c97cd67372.tif"
fp_meta = r"/home/geoai/geoai_data_test2/data_npark_prod_PlanetScope/data/2022/03/images/9adb4c6de91945c0812644280ea18160.tif"

In [5]:
with rasterio.open(fp) as src:
    mask_nodata = np.array([src.read_masks(1)])/255
with rasterio.open(fp_meta) as src:
    meta=src.meta

In [6]:
with rasterio.open(r"/home/geoai/geoai_data_test2/data_npark_prod_PlanetScope/xoa.tif",'w',**meta) as dst:
    dst.write(mask_nodata.astype('uint8'))